# Moodle Database: Educational Data Log Analysis 
## Overview
This week we analyse the 2019 10 Academy learners activity in the Moodle Learning Management System. The moodle LMS is a highly scalable framework, and all students activities are stored in a highly structured database.  
## Business Need
Many educational facilities such as colleges, universities, bootcamps rely on scalable and versatile Learning Management Systems. 

10 Academy, like any other educational facility, is interested to understand the learners skill and knowledge development, and is interested to build models that are able to predict possible dropouts as well as classify learners into doing, well, doing ok, and struggling groups. 10 Academy is also interested in automating the process of reminding learners approaching deadlines, providing quick feedback based on their overall community engagement and performance. Moreover, given the main goal of 10 Academy training is to make students job ready, it wants to measure students' performance across many relevant metrics. 

**Challenge:** to explore the 10 Academy Moodle logs stored in the database together with many other relevant tables. By the end of analysis, we build a Tableau dashboard that illustrates the progress of students across time.

### Most Important Tables (MIT)

Moodle database is complex - with more than 400 connected tables! In this project we are interested only in the subset of the tables. The most important tables we will consider in this challenge are (tables in bold are VIP)
* **mdl_logstore_standard_log**
* **mdl_context**
* **mdl_user**
* **mdl_course**
* **mdl_modules** 
* **mdl_course_modules**
* **mdl_course_modules_completion** 
* mdl_grade_items
* **mdl_grade_grades**
* mdl_grade_categories
* mdl_grade_items_history
* mdl_grade_grades_history
* mdl_grade_categories_history
* mdl_forum
* mdl_forum_discussions
* mdl_forum_posts

The tasks are divided into four sub-tasks
1. Moodle database schema understanding
2. Data Extraction Transformation and Loading (ETL)
3. Dashboard making with Tableau
4. Automation and Deployment 


## Task 1: Moodle Database Schema Understanding
The first task focuses on having a clear understanding of the moodle database schema.

We'll start with importing the necessary libraries to used for this analysis

In [198]:
import psycopg2
import pandas as pd
import numpy as np
import datetime

Then we create a connection to our database using psycopg2

In [2]:
conn = psycopg2.connect(host = '127.0.0.1',
                       database = 'moodle',
                       user = 'postgres',
                       password = '@wamaitha1')

cur = conn.cursor()

We then proceed to understand the moodle database schema by writing SQL scripts to compute the following:

#### The number of tables

In [3]:
cur.execute('SELECT COUNT(*) FROM information_schema.tables')

rows = cur.fetchall()

rows

[(636,)]

#### The number of records in each of the tables given in the MIT section

In [4]:
def table_rows(tables):
    row_num = []
    for table in tables:
        cur.execute('select count(*) from {t}'.format(t=table))
        rows = cur.fetchall()
        row_num.append([table, rows])
    
    tables_df = pd.DataFrame(row_num, columns = ['Table', 'Number of rows'])
    return tables_df

In [5]:
table_rows(['mdl_logstore_standard_log', 'mdl_context', 'mdl_user', 'mdl_course', 'mdl_modules', 'mdl_course_modules', 'mdl_course_modules_completion', 'mdl_grade_items', 'mdl_grade_grades', 'mdl_grade_categories', 'mdl_grade_items_history', 'mdl_grade_grades_history', 'mdl_grade_categories_history', 'mdl_forum', 'mdl_forum_discussions', 'mdl_forum_posts'])

,Table,Number of rows
0,mdl_logstore_standard_log,"[(417554,)]"
1,mdl_context,"[(4359,)]"
2,mdl_user,"[(1052,)]"
3,mdl_course,"[(15,)]"
4,mdl_modules,"[(26,)]"
5,mdl_course_modules,"[(290,)]"
6,mdl_course_modules_completion,"[(4483,)]"
7,mdl_grade_items,"[(113,)]"
8,mdl_grade_grades,"[(3643,)]"
9,mdl_grade_categories,"[(16,)]"


#### Number of quiz submissions by hour of day

In [6]:
cur.execute("select count(id), EXTRACT(HOUR FROM to_timestamp(timecreated)) AS HOUR from mdl_logstore_standard_log where action='submitted' AND component='mod_quiz' group by EXTRACT(HOUR FROM to_timestamp(timecreated))")

rows = cur.fetchall()

data = pd.DataFrame(rows, columns = ['Number of quiz submission', 'Hour of day']).set_index('Hour of day')

data

,Number of quiz submission
Hour of day,
0.0,26
1.0,23
2.0,30
3.0,29
4.0,7
5.0,14
6.0,10
7.0,18
8.0,17


#### Monthly usage time of learners who have confirmed and are not deleted

In [7]:
#we start with getting the data from the database applying all the condition needed
#that is fetching the data of only students who are confimed and not deleted
#we'll also fetch the 'timecreated' column as a timestamp for easy calculation and extract the month from the timemstamp
cur.execute("select u.id, to_timestamp(l.timecreated), EXTRACT(MONTH FROM to_timestamp(l.timecreated)) as Month from mdl_user u, mdl_logstore_standard_log l where u.id = l.userid and u.confirmed=1 and u.deleted=0")
rows = cur.fetchall()
df = pd.DataFrame(rows, columns=['userid', 'timecreated', 'Month'])

#the created month column takes up the float64 datatype, i.e 1.0, therefore, we can change this format to 'int64' so that its only 1 and not 1.0
df['Month'] = df['Month'].astype('int64')

#then we finish by grouping our data based on our students and the month, get the maximum and minimum time per month and subtract the two to get approximate monthly usage time per student.
df1 = df.groupby(['userid','Month']).agg({'timecreated': ['max', 'min']})
df1.columns = ['Maximum Time', 'Minimum Time']
df1['Monthly Usage Time'] = df1['Maximum Time'] - df1['Minimum Time']

In [8]:
# we can inspect the generated data, to confirm that we have the data as required
df1.head(30)

Maximum Time              Minimum Time  \
userid Month                                                       
1      1     2019-01-25 19:52:51+03:00 2019-01-25 19:52:31+03:00   
2      1     2019-01-31 16:36:42+03:00 2019-01-24 09:25:48+03:00   
       2     2019-02-28 20:03:35+03:00 2019-02-01 23:30:02+03:00   
       3     2019-03-31 08:00:03+03:00 2019-03-01 14:03:01+03:00   
       4     2019-04-30 23:28:35+03:00 2019-04-01 08:00:05+03:00   
       5     2019-05-31 19:00:02+03:00 2019-05-01 02:30:02+03:00   
       6     2019-06-29 00:00:03+03:00 2019-06-01 08:00:05+03:00   
       7     2019-07-29 14:00:03+03:00 2019-07-01 16:30:04+03:00   
       8     2019-08-01 03:30:03+03:00 2019-08-01 03:30:03+03:00   
3      1     2019-01-31 16:25:15+03:00 2019-01-25 16:54:58+03:00   
       2     2019-02-28 23:45:09+03:00 2019-02-06 16:37:31+03:00   
       3     2019-03-27 19:40:35+03:00 2019-03-01 00:53:25+03:00   
       4     2019-04-30 21:14:48+03:00 2019-04-04 14:01:34+03:00   
       5     2019-05-31 16:44:31+03:00 2019-05-01 12:14:05+03:00   
       6     2019-06-25 15:01:03+03:00 2019-06-02 11:41:11+03:00   
       7     2019-07-31 21:25:08+03:00 2019-07-01 16:35:35+03:00   
5      1     2019-01-31 16:23:58+03:00 2019-01-25 20:22:03+03:00   
       2     2019-02-23 16:25:42+03:00 2019-02-07 16:10:55+03:00   
       3     2019-03-30 13:38:37+03:00 2019-03-11 20:08:35+03:00   
       4     2019-04-30 14:42:26+03:00 2019-04-01 17:31:41+03:00   
       5     2019-05-26 16:45:26+03:00 2019-05-25 22:02:33+03:00   
       7     2019-07-31 21:25:08+03:00 2019-07-01 08:44:29+03:00   
7      1     2019-01-26 12:54:29+03:00 2019-01-26 12:52:35+03:00   
       2     2019-02-22 13:53:07+03:00 2019-02-20 22:33:57+03:00   
       3     2019-03-27 18:57:16+03:00 2019-03-11 20:08:22+03:00   
       7     2019-07-22 16:22:13+03:00 2019-07-08 06:55:52+03:00   
8      1     2019-01-31 10:46:34+03:00 2019-01-31 10:44:13+03:00   
       2     2019-02-22 13:53:20+03:00 2019-02-20 22:33:57+03:00   
       3     2019-03-27 18:57:22+03:00 2019-03-27 14:31:32+03:00   
9      2     2019-02-26 16:58:35+03:00 2019-02-07 13:59:08+03:00   

             Monthly Usage Time  
userid Month                     
1      1        0 days 00:00:20  
2      1        7 days 07:10:54  
       2       26 days 20:33:33  
       3       29 days 17:57:02  
       4       29 days 15:28:30  
       5       30 days 16:30:00  
       6       27 days 15:59:58  
       7       27 days 21:29:59  
       8        0 days 00:00:00  
3      1        5 days 23:30:17  
       2       22 days 07:07:38  
       3       26 days 18:47:10  
       4       26 days 07:13:14  
       5       30 days 04:30:26  
       6       23 days 03:19:52  
       7       30 days 04:49:33  
5      1        5 days 20:01:55  
       2       16 days 00:14:47  
       3       18 days 17:30:02  
       4       28 days 21:10:45  
       5        0 days 18:42:53  
       7       30 days 12:40:39  
7      1        0 days 00:01:54  
       2        1 days 15:19:10  
       3       15 days 22:48:54  
       7       14 days 09:26:21  
8      1        0 days 00:02:21  
       2        1 days 15:19:23  
       3        0 days 04:25:50  
9      2       19 days 02:59:27

#### Count of log events per user for the following verbs: ['loggedin', 'viewed', 'started', ,'submitted', 'uploaded', 'updated', 'searched', 'resumed', 'answered', 'attempted', 'abandoned']

In [27]:
def log_events(events):
    cur.execute("select id from mdl_user order by id")
    rows = cur.fetchall()
    df = pd.DataFrame(rows, columns = ['userid'])
    for event in events:
        cur.execute("Select userid, count(*) from mdl_logstore_standard_log where action = '{e}' group by userid".format(e=event))
        rows = cur.fetchall()
        df1 = pd.DataFrame(rows, columns = ['userid', event])
        df = df.merge(df1, on = ['userid'], how = 'outer').fillna(0)
        
    events_log = df.copy().set_index('userid')
    return events_log

In [30]:
log_events(['loggedin', 'viewed', 'started', 'submitted', 'uploaded', 'updated', 'searched', 'resumed', 'answered', 'attempted', 'abandoned'])

,loggedin,viewed,started,submitted,uploaded,updated,searched,resumed,answered,attempted,abandoned
userid,,,,,,,,,,,
1,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
2,169.0,2492.0,14.0,21.0,21.0,2551.0,0.0,0.0,0.0,0,5.0
3,107.0,4041.0,17.0,2.0,6.0,1467.0,0.0,4.0,0.0,0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
5,54.0,694.0,11.0,5.0,1.0,19.0,0.0,2.0,2.0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
1049,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
1050,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
1051,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0


#### Python class to pull: overall grade of learners and number of forum posts

In [13]:
class GradeForums:
    def _init_(self):
        '''
        a python class that pulls the overall grade of learners with the grade() method and the number of forum posts with the forums() method.
        both methods return a dataframe with the information pulled fromm the database.
        '''
        
    def grade():
        cur.execute('select userid, sum(finalgrade) from mdl_grade_grades group by userid')
        rows = cur.fetchall()
        df = pd.DataFrame(rows, columns = ['userid', 'overall grade'])
        return df
    
    def forums():
        cur.execute('select count(*) from mdl_forum_posts')
        rows = cur.fetchall()
        df = pd.DataFrame(rows, columns = ['Number of forum posts'])
        return df

In [14]:
gf = GradeForums
gf.grade()

,userid,overall grade
0,351,418.00000
1,87,200.00000
2,184,200.00000
3,116,218.66666
4,273,200.00000
...,...,...
402,130,402.00000
403,23,200.00000
404,270,224.00000
405,846,2345.80954


In [15]:
gf.forums()

,Number of forum posts
0,131


## Task 2: Data Extraction Transformation and Loading (ETL)
**Write a python class to perform the following:**
* Compute dedication time
* Compute login and activity counts
* Based on the following metrics, group students as top 1%, 5%, 10%, 25%
1. Login count
2. Activity count
3. Dedication time

In [199]:
class ExtractionTransformationLoading:
    def _init_():
        '''
        a python class to compute the dedication time of students,
        their login and activity counts and
        groups the students as top 1%, 5%, 10%, and 25% based on login count, activity count, and dedication time.
        
        login_activity_count: a function that fetches the login and activity count of students from the log table in the moodle database.
        '''
        
    def dedication_time(self):
        '''
        function to compute the dedication time of a student.
        
        Derivation of the dedication time has been based on the concept of sessions and session duration.
        Each session has a beginning, signified by a login event, a series of events and the last activity before the next login.
        The session duration, on the other hand, is the time between the last activity of the session and the login time of the session.
            Each session duration has been equated to a maximum of 4 hours.
        The duration time is then computer by summing up all session durations per student.
        
        Returns: a dataframe with the user and their dedication time.
        '''
        
        #we start with creating a list of all users derived from the mdl_users table excluding the negative user ids
        cur.execute("select distinct(userid) from mdl_logstore_standard_log where userid != -10 and userid != -1 order by userid")
        rows = cur.fetchall()
        df = pd.DataFrame(rows, columns = ['userid'])
        users = df.userid.values
        
        #then we fetch time data for each action and user from the log table
        cur.execute("select userid, to_timestamp(timecreated), action from mdl_logstore_standard_log where userid != -10 and userid != -1 order by userid, timecreated")
        rows = cur.fetchall()
        df1 = pd.DataFrame(rows, columns = ['userid', 'timecreated', 'action'])
        
        #since we will be sessions, we create an empty column for the sessions
        df1['Sessions'] = ''
        
        #then we assign a session number to each login activity for each user.
        #The session number for each user starts at 1 as indicated below
        for user in users:
            session = 1
            for i in range(0,len(df1)):
                if (df1.loc[i, 'userid'] == user) and ((df1.loc[i, 'action'] == 'loggedin') or (df1.loc[i, 'action'] == 'loggedinas')):
                    df1.loc[i, 'Sessions']= 'Session ' + str(session)
                    session+=1
        
        #we then proceed to populate all the activities between the session starts with the session above it.
        for i in range(0,len(df1)):
            if (df1.loc[i, 'Sessions'] == ''):
                #this step is done to cater for any first user without a login as the first activity
                if i == 0:
                    df1.loc[i, 'Sessions']= 'Session 0'
                else:
                    df1.loc[i, 'Sessions']= df1.loc[i-1, 'Sessions']
        
        #now that we have all our activities  assigned to a Session, we group the data based on the user and a Session 
        #and calculate the duration for each session (Session Duration))
        df2 = df1.groupby(['userid', 'Sessions']).agg({'timecreated': ['max', 'min']})
        df2.columns = ['Max Session Time', 'Min Session Time']
        df2['Session Duration'] = df2['Max Session Time'] - df2['Min Session Time']
        df2 = df2.reset_index()
        
        #then we proceed to create a session duration time constrain where the maximum session duration is set to 4 hours
        max_session_time = datetime.timedelta(hours=4)
        
        #any session duration that exceed the maximum duration is then set to the maximum duration
        for i in range(0,len(df2)):
            if (df2.loc[i, 'Session Duration'] > max_session_time):
                df2.loc[i, 'Session Duration']= max_session_time
        
        #finally, we group again our data based on the user and sum all the Session Durations to get the dedication time of the user.
        df3 = df2.groupby(['userid']).agg({'Session Duration': 'sum'})
        df3.columns = ['Dedication Time']
        return df3
        
    def login_activity_count(self):
        '''
        a function to fetch the login and activity count of students.
        Returns: a dataframe with the student and their activity and login count
        '''
        operators = ['=', '!=']
        counts = ['Login Count', 'Activity Count']
        
        #we start with fetching all the students from mdl_user table by their id
        cur.execute("Select id from mdl_user order by id")
        rows = cur.fetchall()
        df = pd.DataFrame(rows, columns=['userid'])
        
        for operator,count in zip(operators, counts):
            cur.execute("Select userid, count(*) from mdl_logstore_standard_log where action {o} 'loggedin' or action {o} 'loggedinas' group by userid".format(o=operator))
            rows = cur.fetchall()
            df1 = pd.DataFrame(rows, columns = ['userid', count])
            df = (df.merge(df1, on = ['userid'], how = 'outer').fillna(0)).set_index('userid')
        return df

In [ ]:
lac = ExtractionTransformationLoading()
dedication_time_df = lac.dedication_time()

In [195]:
df

,Login Count,Activity Count
userid,,
1,1.0,3.0
2,175.0,45023.0
3,107.0,12922.0
4,0.0,3.0
5,54.0,1079.0
...,...,...
1051,0.0,9.0
1052,0.0,9.0
-10,0.0,1.0


In [ ]:
login_activity_df = lac.login_activity_count()

In [ ]:
student_analysis_df = (login_activity_df.merge(dedication_time_df, on = ['userid'], how = 'outer').fillna(0)).set_index('userid')

def top_by_activity():
    activities = ['Login Count', 'Activity Count', 'Dedication Time']
    names = ['login', 'activity', 'dedication']
    for activity, name in zip(activities, names):
        student_analysis_df = student_analysis_df.sort_values(by=[activity], ascending=False)
    
        df_length = len(student_analysis_df)
    
        percentages = [0.01, 0.05, 0.10, 0.25]
        top = ['1%', '5%', '10%', '25%']
        for p, t in zip(percentages, top):
            top_data = student_analysis_df.head(int(np.round(p*df_length)))
            top_data.to_csv('top_{n}_{t}.csv'.format(n=name, t=t), index=False)